<a href="https://colab.research.google.com/github/aviMitra/EVA5/blob/main/Assignment%204/Assignment_4_MNIST_Network_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [40]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        set_bias = False
        dropout = 0.15
        #Block 1
        self.conv1 = nn.Conv2d(1, 18, 3, bias=set_bias)   # 1x28x28  -> 18x26x26, RF: 3
        self.batch1 = nn.BatchNorm2d(18)
        self.drop1 = nn.Dropout2d(dropout)
        self.conv2 = nn.Conv2d(18, 36, 3, bias=set_bias)  # 18x26x26 -> 36x24x24, RF: 5
        self.batch2 = nn.BatchNorm2d(36)
        self.drop2 = nn.Dropout2d(dropout)

        self.pool1 = nn.MaxPool2d(2, 2)                   # 36x24x24 -> 36x12x12, RF: 10
        self.ant1 = nn.Conv2d(36, 20, 1, bias=set_bias)   # 36x12x12 -> 20x12x12, RF: 10 
        
        # Block 2
        self.conv3 = nn.Conv2d(20, 35, 3, bias=set_bias)  # 20x12x12 -> 35x10x10, RF: 12
        self.batch4 = nn.BatchNorm2d(35)
        self.drop4 = nn.Dropout2d(dropout)
        self.pool2 = nn.MaxPool2d(2, 2)                   # 35x10x10 -> 35x5x5,   RF: 24
        self.ant2 = nn.Conv2d(35, 20, 1, bias=set_bias)   # 35x5x5   -> 20x5x5,   RF: 24

        # Block 3
        self.conv5 = nn.Conv2d(20, 32, 3, bias=set_bias)  # 20x5x5   -> 32x3x3  , RF: 26
        self.ant3 = nn.Conv2d(32, 10, 1, bias=set_bias)   # 32x3x3   -> 10x3x3  , RF: 26
        self.out = nn.AvgPool2d(3,3)

    def forward(self, x):
        # Block 1
        x = self.drop1(self.batch1(F.relu(self.conv1(x))))
        x = self.drop2(self.batch2(F.relu(self.conv2(x))))
        x = self.ant1(self.pool1(x))

        # Block 2
        x = self.drop4(self.batch4(F.relu(self.conv3(x))))
        x = self.ant2(self.pool2(x))

        # Block 3
        x = self.ant3(self.conv5(x))
        x = self.out(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=1)

In [41]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 18, 26, 26]             162
       BatchNorm2d-2           [-1, 18, 26, 26]              36
         Dropout2d-3           [-1, 18, 26, 26]               0
            Conv2d-4           [-1, 36, 24, 24]           5,832
       BatchNorm2d-5           [-1, 36, 24, 24]              72
         Dropout2d-6           [-1, 36, 24, 24]               0
         MaxPool2d-7           [-1, 36, 12, 12]               0
            Conv2d-8           [-1, 20, 12, 12]             720
            Conv2d-9           [-1, 35, 10, 10]           6,300
      BatchNorm2d-10           [-1, 35, 10, 10]              70
        Dropout2d-11           [-1, 35, 10, 10]               0
        MaxPool2d-12             [-1, 35, 5, 5]               0
           Conv2d-13             [-1, 20, 5, 5]             700
           Conv2d-14             [-1, 3

In [42]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [43]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * (correct / len(test_loader.dataset))))
    return test_loss

In [46]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=2)


for epoch in range(19):
    print('########################### Epoch No.: ', str(epoch+1), '######################################')
    train(model, device, train_loader, optimizer, epoch)
    loss = test(model, device, test_loader)
    scheduler.step(loss)
    print('Learning rate: ', optimizer.param_groups[0]['lr'])



########################### Epoch No.:  1 ######################################

Test set: Average loss: 0.0696, Accuracy: 9776/10000 (97.76%)

Learning rate:  0.1
########################### Epoch No.:  2 ######################################

Test set: Average loss: 0.0548, Accuracy: 9823/10000 (98.23%)

Learning rate:  0.1
########################### Epoch No.:  3 ######################################

Test set: Average loss: 0.0407, Accuracy: 9873/10000 (98.73%)

Learning rate:  0.1
########################### Epoch No.:  4 ######################################

Test set: Average loss: 0.0343, Accuracy: 9891/10000 (98.91%)

Learning rate:  0.1
########################### Epoch No.:  5 ######################################

Test set: Average loss: 0.0433, Accuracy: 9843/10000 (98.43%)

Learning rate:  0.1
########################### Epoch No.:  6 ######################################

Test set: Average loss: 0.0342, Accuracy: 9891/10000 (98.91%)

Learning rate:  0.1
##########